In [ ]:
!pip install casadi
!pip install celluloid #convenient package for making animations

In [2]:
import time
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
import pdb
from random import sample
                    
from Simulators import Simulator
from SMPC import SMPC_MMPreds

In [ ]:
## Run SMPC with first policy type
## OL: Open-loop, IA: Interaction-Aware
Policy=["SMPC_OL", "SMPC", "SMPC_IA", "SMPC_OBCA", "SMPC_IA_OBCA"]
pol_num=1   #choose which policy to use

## TI: Traffic Intersection, HW: Highway
Env=["TI", "HW"]
env_num=0   #choose which environment to use

## True mode for the scene. 
mode=0      #For TI, modes go from 0 to 3. For HW, modes go from 0 to 1



## Flag for choosing whether to continue or stop when a collision occurs
stop_after_collision=True

## Flag for choosing what control to apply if MPC is feasible. 
## By default, the previous input is applied
break_if_infeasible=False


Sim=Simulator(MODE=mode, ENV=Env[env_num], EV_POLICY_TYPE=Policy[pol_num])
smpc = SMPC_MMPreds(DT=Sim.dt, NOISE_STD=Sim.noise_std, O_const1=Sim.o_c1, O_const2=Sim.o_c2, 
                    N_modes=Sim.N_modes,N_TV=Sim.N_TV, TV_GAINS=Sim.gains, IA_GAINS=Sim.IA_gains, TV_SPEED=Sim.tv_v, TV_L=Sim.tv_l,
                    POLICY=Policy[pol_num], ENV=Env[env_num])


is_feas=False
feas_list=[]


collision_detected=False
ca_list=[]

while not Sim.done() and not (collision_detected and stop_after_collision):
    if Sim.t==0 or not is_feas:
        update_dict=Sim.get_update_dict(smpc.N)
    else:
        if "OBCA" not in Policy[pol_num]:
            update_dict=Sim.get_update_dict(smpc.N, sol_dict['h_opt'], sol_dict['M_opt'], sol_dict['K_opt'], sol_dict['nom_z_tv'])
        else:
            update_dict=Sim.get_update_dict(smpc.N, sol_dict['h_opt'], sol_dict['M_opt'], sol_dict['K_opt'], sol_dict['lmbd_opt'], sol_dict['nu_opt'], sol_dict['nom_z_tv'])
    
    smpc.update(update_dict)
    sol_dict=smpc.solve()
    is_feas=sol_dict['feasible']
    if not is_feas and break_if_infeasible:
        Sim.run_step(smpc.A_MIN)
        u_applied=smpc.A_MIN
    else:
        Sim.run_step(sol_dict['u_control'])
        u_applied=sol_dict['u_control']
    print("Feasible:",is_feas, ", Time:", Sim.t)
    print("EV states (x,y,v): ", Sim.ev_traj[:,Sim.t],  ", EV input (a): ", u_applied)
    if env_num==0:
        print("TV states[0] (x,y,v): ", Sim.tv_traj[0][:,Sim.t], ", TV states[1] (x,y,v):", Sim.tv_traj[1][:,Sim.t])        
    else:
        print("TV states[0] (x,y,psi):", Sim.tv_traj[0][:,Sim.t])
    print("-----------------------------------------------------")
    collision_detected=Sim.check_collision()
    ca_list.append(collision_detected)
    feas_list.append(is_feas)
   
    

    


In [ ]:
## Run SMPC with second policy type
Policy=["SMPC_OL", "SMPC", "SMPC_IA", "SMPC_OBCA", "SMPC_IA_OBCA"]
pol_num=0

Sim2=Simulator(MODE=mode, ENV=Env[env_num], EV_POLICY_TYPE=Policy[pol_num])
smpc2 = SMPC_MMPreds(DT=Sim2.dt, NOISE_STD=Sim2.noise_std, O_const1=Sim2.o_c1, O_const2=Sim2.o_c2, 
                    N_modes=Sim2.N_modes,N_TV=Sim2.N_TV, TV_GAINS=Sim2.gains, IA_GAINS=Sim2.IA_gains, TV_SPEED=Sim2.tv_v, TV_L=Sim2.tv_l,
                    POLICY=Policy[pol_num], ENV=Env[env_num])
is_feas=False
feas_list2=[]

collision_detected=False
ca_list2=[]

while not Sim2.done() and not (collision_detected and stop_after_collision):
    if Sim2.t==0 or (not is_feas):
        update_dict=Sim2.get_update_dict(smpc2.N)
    else:
        if "OBCA" not in Policy[pol_num]:
            update_dict=Sim2.get_update_dict(smpc2.N, sol_dict['h_opt'], sol_dict['M_opt'], sol_dict['K_opt'], sol_dict['nom_z_tv'])
        else:
            update_dict=Sim2.get_update_dict(smpc2.N, sol_dict['h_opt'], sol_dict['M_opt'], sol_dict['K_opt'], sol_dict['lmbd_opt'], sol_dict['nu_opt'], sol_dict['nom_z_tv'])
    
    smpc2.update(update_dict)
    sol_dict=smpc2.solve()
    is_feas=sol_dict['feasible']
    if not is_feas and break_if_feasible:
        Sim2.run_step(smpc2.A_MIN)
        u_applied=smpc2.A_MIN
    else:
        Sim2.run_step(sol_dict['u_control'])
        u_applied=sol_dict['u_control']
    print("Feasible:",is_feas, ", Time:", Sim2.t)
    print("EV states (x,y,v): ", Sim2.ev_traj[:,Sim2.t], ", EV input (a): ", u_applied)
    if env_num==0:
        print("TV states[0] (x,y,v): ", Sim2.tv_traj[0][:,Sim2.t], ", TV states[1] (x,y,v): ", Sim2.tv_traj[1][:,Sim2.t])
    else:
        print("TV states[0] (x,y,psi): ", Sim2.tv_traj[0][:,Sim2.t])

    print("-----------------------------------------------------")
    collision_detected=Sim2.check_collision()
    ca_list2.append(collision_detected)
    feas_list2.append(is_feas)


In [ ]:
## Plotting
%matplotlib inline

import matplotlib.pyplot as plt


fig=plt.figure(figsize=(10, 8))

a1=plt.subplot(311)
plt.plot(Sim.ev_traj[0,:Sim.t], color='g',linestyle='-', label=Sim.ev_pol)
plt.plot(Sim2.ev_traj[0,:Sim2.t], color='r',linestyle='-', label= Sim2.ev_pol)
plt.xlabel('time')
plt.ylabel('x',fontsize=18)
plt.xlim(0,Sim2.t)
a1.set_title("Policy: "+Sim.ev_pol+" vs Policy: "+Sim2.ev_pol+" For ENV: "+Sim.env)
plt.legend()
a2=plt.subplot(312)
plt.plot(Sim.ev_traj[2,:Sim.t], color='g',linestyle='-', label=Sim.ev_pol)
plt.plot(Sim2.ev_traj[2,:Sim2.t], color='r',linestyle='-', label= Sim2.ev_pol)
plt.plot(smpc2.V_MAX*np.ones(Sim2.t), color='k',linewidth=2.0,linestyle='-', label= 'v_max')
plt.plot(smpc2.V_MIN*np.ones(Sim2.t), color='y',linewidth=2.0,linestyle='-', label= 'v_min')
plt.xlabel('time')
plt.ylabel('v',fontsize=18)
plt.xlim(0,Sim2.t)
plt.legend()
a3=plt.subplot(313)
plt.plot(Sim.ev_u[0,1:Sim.t], color='g',linestyle='-', label=Sim.ev_pol)
plt.plot(Sim2.ev_u[0,1:Sim2.t], color='r',linestyle='-', label= Sim2.ev_pol)
plt.plot(smpc2.A_MAX*np.ones(Sim2.t), color='k',linewidth=2.0,linestyle='-', label= 'a_max')
plt.plot(smpc2.A_MIN*np.ones(Sim2.t), color='y',linewidth=2.0,linestyle='-', label= 'a_min')
plt.xlabel('time')
plt.ylabel('a',fontsize=18)
plt.xlim(0,Sim2.t)
plt.legend()

plt.show()

plt.savefig(Sim.ev_pol+'_vs_'+Sim2.ev_pol+'_'+Sim.env+'.png')


In [ ]:
# Make animation for first policy
%matplotlib notebook
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from celluloid import Camera
from IPython.display import HTML

fig= plt.figure()
camera = Camera(fig)

if Env[env_num]=="TI":
    for i in range(Sim.t+1):
        plt.plot([-45, 21], [5, 5], color='k', lw=2)
        plt.plot([-45, 21], [-5, -5], color='k', lw=2)
        plt.plot([41, 50], [5, 5], color='k', lw=2)
        plt.plot([41, 50], [-5, -5], color='k', lw=2)
        plt.plot([21, 21], [5, 30], color='k', lw=2)
        plt.plot([41, 41], [5, 30], color='k', lw=2)
        plt.plot([21, 21], [-5, -30], color='k', lw=2)
        plt.plot([41, 41], [-5, -30], color='k', lw=2)
        plt.plot(Sim.ev_traj[0,:i], Sim.ev_traj[1,:i], color='green', lw=3)
        if not feas_list[i-1]:
            plt.text(-20,20, "INFEASIBLE!", fontsize=8, color='r')
        if ca_list[i-1]:
            plt.text(-20,30, "Collision!", fontsize=15, color='r')
        plt.gca().add_patch(Rectangle((Sim.tv_traj[0][0,i-1]-1.7,Sim.tv_traj[0][1,i-1]-2.9),3.4,5.8,linewidth=1,edgecolor='r',facecolor='none'))
        plt.gca().add_patch(Rectangle((Sim.tv_traj[1][0,i-1]-1.7,Sim.tv_traj[1][1,i-1]-2.9),3.4,5.8,linewidth=1,edgecolor='r',facecolor='none'))
        plt.gca().add_patch(Rectangle((Sim.ev_traj[0,i-1]-2.9,Sim.ev_traj[1,i-1]-1.7),5.8,3.4,linewidth=1,edgecolor='g',facecolor='none'))
        plt.axis('equal')
        plt.title("Policy type : "+Sim.ev_pol)
        camera.snap()
else:
    for i in range(Sim.t+1):
        plt.plot([-40, 60], [12, 12], color='k', lw=2)
        plt.plot([-40, 60], [-5, -5], color='k', lw=2)
        plt.plot([-40, 60], [4, 4], color='k', lw=1)
        plt.plot(Sim.ev_traj[0,:i], Sim.ev_traj[1,:i], color='green', lw=3)
        if not feas_list[i-1]:
            plt.text(-20,20, "INFEASIBLE!", fontsize=10, color='r')
        plt.gca().add_patch(Rectangle((Sim.tv_traj[0][0,i-1]-2.9,Sim.tv_traj[0][1,i-1]-1.7),5.8,3.4,linewidth=1,edgecolor='r',facecolor='none'))
        plt.gca().add_patch(Rectangle((Sim.ev_traj[0,i-1]-2.9,Sim.ev_traj[1,i-1]-1.7),5.8,3.4,linewidth=1,edgecolor='g',facecolor='none'))
        plt.axis('equal')
        plt.title("Policy type : "+Sim.ev_pol)
        camera.snap()
    
animation = camera.animate(repeat = True, repeat_delay = 500)
HTML(animation.to_html5_video())
# animation.save('animation_'+Sim.ev_pol+'_'+Sim.env+'.mp4')


In [ ]:
# Make animation for second policy
%matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from celluloid import Camera
from IPython.display import HTML

fig= plt.figure()
camera = Camera(fig)

if Env[env_num]=="TI":
    for i in range(Sim2.t+1):
        plt.plot([-45, 21], [5, 5], color='k', lw=2)
        plt.plot([-45, 21], [-5, -5], color='k', lw=2)
        plt.plot([41, 50], [5, 5], color='k', lw=2)
        plt.plot([41, 50], [-5, -5], color='k', lw=2)
        plt.plot([21, 21], [5, 30], color='k', lw=2)
        plt.plot([41, 41], [5, 30], color='k', lw=2)
        plt.plot([21, 21], [-5, -30], color='k', lw=2)
        plt.plot([41, 41], [-5, -30], color='k', lw=2)
        plt.plot(Sim2.ev_traj[0,:i], Sim2.ev_traj[1,:i], color='green', lw=3)
        if not feas_list2[i-1]:
            plt.text(-20,20, "INFEASIBLE!", fontsize=10, color='r')
        plt.gca().add_patch(Rectangle((Sim2.tv_traj[0][0,i-1]-1.7,Sim2.tv_traj[0][1,i-1]-2.9),3.4,5.8,linewidth=1,edgecolor='r',facecolor='none'))
        plt.gca().add_patch(Rectangle((Sim2.tv_traj[1][0,i-1]-1.7,Sim2.tv_traj[1][1,i-1]-2.9),3.4,5.8,linewidth=1,edgecolor='r',facecolor='none'))
        plt.gca().add_patch(Rectangle((Sim2.ev_traj[0,i-1]-2.9,Sim2.ev_traj[1,i-1]-1.7),5.8,3.4,linewidth=1,edgecolor='g',facecolor='none'))
        plt.axis('equal')
        plt.title("Policy type : "+Sim2.ev_pol)
        camera.snap()
else:
    for i in range(Sim2.t+1):
        plt.plot([-40, 60], [12, 12], color='k', lw=2)
        plt.plot([-40, 60], [-5, -5], color='k', lw=2)
        plt.plot([-40, 60], [4, 4], color='k', lw=1)
        plt.plot(Sim2.ev_traj[0,:i], Sim2.ev_traj[1,:i], color='green', lw=3)
        if not feas_list2[i-1]:
            plt.text(-20,20, "INFEASIBLE!", fontsize=10, color='r')
        if not ca_list2[i-1]:
            plt.text(-20,30, "Collision!", fontsize=15, color='r')
        plt.gca().add_patch(Rectangle((Sim2.tv_traj[0][0,i-1]-2.9,Sim2.tv_traj[0][1,i-1]-1.7),5.8,3.4,linewidth=1,edgecolor='r',facecolor='none'))
        plt.gca().add_patch(Rectangle((Sim2.ev_traj[0,i-1]-2.9,Sim2.ev_traj[1,i-1]-1.7),5.8,3.4,linewidth=1,edgecolor='g',facecolor='none'))
        plt.axis('equal')
        plt.title("Policy type : "+Sim2.ev_pol)
        camera.snap()
    
animation = camera.animate(repeat = True, repeat_delay = 500)
HTML(animation.to_html5_video())
# M=animation.save('animation2_'+Sim2.ev_pol+'_'+Sim2.env+'.mp4')
